In [1]:
#LEGO script combining all lego data into the file: total_lego_data.csv
import pandas as pd
import numpy as np
import translators as ts
import matplotlib.pyplot as plt


df = pd.read_excel('legodata_final.xlsx', index_col=0)
df = df.reset_index()
df_lego_20_22 = df.iloc[:, 9:15]  #removing empty columns
df_lego_20_22n = df_lego_20_22.drop(columns = ["target"])
df_lego_20_22n = df_lego_20_22n.rename(columns={"Grant Reciever": "Reciever", "Grant amount (in DKK thousand)": "Grant size (DKK)", "Descriptions" : "Description"})
df_lego_20_22n.head()



#downloading 2023 lego results 
df_2023 = pd.read_csv('tabula-list-of-grantees-transformed2023.csv',sep=";", names = ["mixed_info", "amount"])
df_2023.head()

#dissecting pdf after tabula conversion

reciever = []
country = []
dkk = []
description = []

for line_nu, i in enumerate(df_2023.mixed_info): 
    if line_nu % 2 == 0:
        money = df_2023.amount[line_nu]
        money = money.replace(",", "")
        amount = int(money+"000")
        dkk.append(amount)
        strings = i.split(" ")
        if strings[-1].endswith(")") and strings[-1].startswith("("):
            strings[-1] = strings[-1].replace(")", "")
            strings[-1] = strings[-1].replace("(", "")
            country.append(strings[-1])
            reciever.append(" ".join(strings[0:-1]))
        elif strings[-1].endswith(")") and strings[-2].startswith("("):
            strings[-1] = strings[-1].replace(")", "")
            strings[-2] = strings[-2].replace("(", "")
            country.append(strings[-2]+ " "+ strings[-1])
            reciever.append(" ".join(strings[0:-2]))
    elif line_nu % 2 != 0:
        description.append(i)
dflego_2023 = pd.DataFrame(list(zip(reciever, country, description, dkk,)), columns=['Reciever','Country', 'Description', "Grant size (DKK)"])

dflego_2023["Year"] = 2023

total_dataframe = pd.concat([df_lego_20_22n, dflego_2023], axis=0)
len(total_dataframe)
total_dataframe.to_csv('total_lego_data.csv', index=True, header=True, sep=';')
